In [1]:
import keras
import numpy as np
from matplotlib import pyplot as plt
import cv2
from pathlib import Path

In [2]:
paths = ["data/normal", "data/codiv19"]  
imgs = []  
labels = [] 

for idx, path in enumerate(paths):
    path_images = Path(path).glob("**/*.jpg")
    path_images = [str(x) for x in path_images]
    
    for path_image in path_images:
        img = cv2.imread(path_image, 0)  # Leer la imagen en modo de escala de grises
        img = img / 255  # Normalizar la imagen
        imgs.append(cv2.resize(img, (150, 150), cv2.INTER_AREA))  # Redimensionar y añadir a la lista
    
    labels.extend([idx] * len(path_images))  # Añadir etiquetas correspondientes

# Convertir las listas a arreglos de NumPy
imgs = np.array(imgs).reshape(-1, 150, 150, 1)
labels = np.array(labels)

print(f"Images shape: {imgs.shape}")
print(f"Num. of labels: {len(labels)}")

Images shape: (30, 150, 150, 1)
Num. of labels: 30


In [3]:
model = keras.Sequential([
    keras.layers.Input((150, 150, 1)),
    #keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
    keras.layers.MaxPooling2D(2, 2),  

    keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    keras.layers.MaxPooling2D(2, 2),  

    #keras.layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    #keras.layers.MaxPooling2D(2, 2),
    
    keras.layers.Dropout(0.5),  
    keras.layers.Flatten(),  
    keras.layers.Dense(64, activation="relu"),

    keras.layers.Dense(2, activation="softmax")
])

# Imprimir el resumen del modelo
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 75, 75, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 37, 37, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43808)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,803,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,804,226 (10.70 MB)

 Trainable params: 2,804,226 (10.70 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    imgs, labels,
    epochs=20,  
    batch_size=32
)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 782ms/step - accuracy: 0.6667 - loss: 0.6740
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 3.1948e-06
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms

In [5]:
#from keras.models import load_model
model.save('modelo.keras')

# El siguiente es el que se convertira con tensorflowjs
# model.save('modelo.h5')

In [6]:
#Cargar el modelo ya entrenado con extensión .keras
#modelo_cargado = load_model('modelo.keras')

In [7]:

# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    # Cargar la imagen
    img = cv2.imread(image_path, 0)  # 0 para leer en escala de grises

    # Convertir la imagen a escala de grises si tiene 3 canales (RGB)
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen a (150, 150)
    img = cv2.resize(img, (150, 150))
    # Normalizar la imagen
    img = img / 255.0
    # Añadir una dimensión para los canales (la imagen debe tener forma (150, 150, 1))
    img = np.expand_dims(img, axis=-1)
    # Añadir una dimensión para que la forma sea (1, 150, 150, 1)
    img = np.expand_dims(img, axis=0)
    return img

# Función para hacer predicciones
def predict_image(model, image_path):
    # Cargar y preprocesar la imagen
    img = load_and_preprocess_image(image_path)
    # Realizar la predicción
    predictions = model.predict(img)
    # Obtener la clase con mayor probabilidad
    predicted_class = np.argmax(predictions)

    # Obtener las probabilidades de cada clase
    probabilities = predictions[0]  # predictions es un array de forma (1, 2)

    # Imprimir el resultado con probabilidades
    print(f"Probabilidad de normal: {probabilities[0]:.4f}")
    print(f"Probabilidad de COVID-19: {probabilities[1]:.4f}")

    # Imprimir el resultado
    if predicted_class == 0:
        print("La imagen es de una persona normal (sin COVID-19).")
    else:
        print("La imagen es de una persona con COVID-19.")




In [8]:
# Ejemplo de uso:
image_path = 'data/normal/N4.jpg'
predict_image(model, image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Probabilidad de normal: 1.0000
Probabilidad de COVID-19: 0.0000
La imagen es de una persona normal (sin COVID-19).
